In [ ]:
fig, ax = plt.subplots()
D = [monitor_sml[:,2], monitor_big[:,2]]
b = ax.boxplot(D)

ax.set_xlabel('Model')
ax.set_ylabel('Reward')

ax.set_xticklabels(['BRS small params', 'BRS big params'])

# plt.title('Reward: 1-small values; 2-big values')

In [ ]:
STD = 3 #1e-3
V_RANGE = (1, 10, 1)
N_SAMPLE_RANGE = (1, N_ROLLOUTS, 1)

env = gym.make('Pendulum-v1', g=9.82)
n_action = 1 
n_state = 3

monitor_rs = []
saved_polices = []
for n_samples in tqdm(np.arange(*N_SAMPLE_RANGE)):
    for v in tqdm(np.arange(*V_RANGE), leave=False):
        model = BRS_RS(n_state, n_action, std=STD, v=v, n_samples=n_samples)
        
        trained_model = train(env, model, STD, v)
        score = evaluate(env, trained_model)
        
        monitor_rs.append([n_samples, v, score])
        saved_polices += [model.policy.M]

monitor_rs = np.array(monitor_rs)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ax.scatter3D(monitor_rs[:,0], monitor_rs[:,1], monitor_rs[:,2], c=monitor_rs[:,2], s=45, alpha=1)
# ax.view_init(10, 90)
ax.set_xlabel('Num of samples per update')
ax.set_ylabel('V')
ax.set_zlabel('Reward')
# ax.plot(monitor_rs[:,0], monitor_rs[:,1], monitor_rs[:,2], color='k')
plt.show()


In [ ]:
fig, ax = plt.subplots()
D = [monitor_sml[:,2], monitor_big[:,2], monitor_rs[:,2]]
b = ax.boxplot(D)

ax.set_xlabel('Model')
ax.set_ylabel('Reward')

ax.set_xticklabels(['BRS small params', 'BRS big params', 'BRS-RS'])

plt.show()

In [ ]:
STD = 1e-3
V_RANGE = (1, 10, 1)
N_SAMPLE_RANGE = (1, N_ROLLOUTS, 1)

env = gym.make('Pendulum-v1', g=9.82)
n_action = 1 
n_state = 3

monitor_ss = []
#saved_polices = []
for n_samples in tqdm(np.arange(*N_SAMPLE_RANGE)):
    for v in tqdm(np.arange(*V_RANGE), leave=False):
        model = BRS_SS(n_state, n_action, std=STD, v=v, n_samples=n_samples)
        
        trained_model = train(env, model, STD, v)
        score = evaluate(env, trained_model)
        
        monitor_ss.append([n_samples, v, score])
        saved_polices += [model.policy.M]

monitor_ss = np.array(monitor_ss)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ax.scatter3D(monitor_rs[:,0], monitor_ss[:,1], monitor_ss[:,2], c=monitor_ss[:,2], s=45, alpha=1)
# ax.view_init(10, 90)
ax.set_xlabel('Rollouts')
ax.set_ylabel('V')
ax.set_zlabel('Reward')
# ax.plot(monitor_rs[:,0], monitor_rs[:,1], monitor_rs[:,2], color='k')
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
D = [monitor_sml[:,2], monitor_big[:,2], monitor_rs[:,2], monitor_ss[:,2]]

box_plot = ax.boxplot(D)

ax.set_xlabel('Model')
ax.set_ylabel('Reward')

ax.set_xticklabels(['BRS small params', 'BRS big params', 'BRS-RS', 'BRS-SS'])

plt.show()

In [ ]:
class ARS:
    def __init__(self, n_state, n_action, b=2, alpha=1e-3, std=1, v=1e-3,
                state_high=None, state_low=None):
        self.policy = Linear_model(n_state, n_action)
        self.buffer = []
        self.alpha = alpha
        self.std = std
        self.v = v
        # b - number of b top-performing directions 
        self.b = b
        
        self.state_low = state_low 
        self.state_high = state_high
        
    def learn(self):
        # sort noise permutations by reward they achive, 
        # and select only high reward ones
        b_buffer = self.sort_directions()
        
        reward_std = b_buffer[:, -2:].std()
        norm = self.alpha / (self.b * reward_std)
        
        reward_sum = 0
        for step in b_buffer:
            # ( R+ - R- ) * noise
            reward_sum += (step[-2] - step[-1]) * step[-3]
    
        self.policy.M = self.policy.M + (norm * reward_sum)
        
        # reset buffer 
        self.buffer = []
    
    def sort_directions(self):
        buffer = np.array(self.buffer, dtype=object)
        b_rewards = buffer[:, -2:].sum(-1)
        
        # idxs from low to high
        b_idxs = np.argsort(b_rewards)
        b_buffer = buffer[b_idxs][-self.b:]
        
        # print(buffer[b_idxs][:-self.b][:,-2:].mean(), buffer[b_idxs][-self.b:][:,-2:].mean())
        return b_buffer
    
    def get_actions(self, state, is_train=False):
        if not self.state_high is None:
            # MINMAX NORM {value - min}/{max - min}
            state = (state - self.state_low)/(self.state_high - self.state_low)
            
        if is_train:
            noise = np.random.normal(0,self.std, size=self.policy.M.shape)
            pos_action = self.policy(state,  self.v, noise)
            neg_action = self.policy(state, -self.v, noise)
            
            pos_action = self._remap_actions(pos_action)
            neg_action = self._remap_actions(neg_action)
            return pos_action, neg_action, noise
        else:
            action = self.policy(state)
            return self._remap_actions(action)
            
    
    def remember(self, memory):
        self.buffer.append(memory)
        
    def _remap_actions(self, action):
        return 2 * np.tanh(action)
    


In [ ]:
import gym
env = gym.make('Pendulum-v1', g=9.82)
env.seed(SEED)
observation = env.reset()

state_high = env.observation_space.high
state_low = env.observation_space.low
model = ARS(3, 1, b=N_ROLLOUTS, std=1, 
            state_low=state_low, state_high=state_high)

REWARDS = []
t_reward = 0
for step in tqdm(range(TOTAL_STEPS)):
    #env.render()
    
    for ro in range(N_ROLLOUTS):
        pos_action, neg_action, noise = model.get_actions(observation, is_train=True)
        pos_observation, pos_reward, pos_done, pos_info = env.step(pos_action)
        neg_observation, neg_reward, neg_done, neg_info = env.step(neg_action)

        model.remember([pos_action[0], neg_action[0], noise, pos_reward, neg_reward])
        t_reward += (pos_reward + neg_reward)/2
        observation = env.reset()
        #observation = pos_observation if np.random.rand()>0.5 else neg_observation
    
    # if (pos_done or neg_done):
        
        
    # if (step % N_ROLLOUTS == 0 and step != 0):
    model.learn()
    REWARDS.append(t_reward)
    t_reward = 0
    
env.close()

In [ ]:
model.policy.M

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(REWARDS)
plt.title(np.mean(REWARDS).round(1))
plt.show()

In [ ]:
meta_data_collector = {''}


env = gym.make('Pendulum-v1', g=9.81)

np.random.seed(SEED)
env.seed(SEED)

observation = env.reset()

state_high = env.observation_space.high
state_low = env.observation_space.low

# EVALUATION
# env = gym.make('Pendulum-v1', g=9.82)
# observation = env.reset()

score = 0
for step in tqdm(range(EVAL_STEPS)):
    #env.render()
    
    action = model.get_actions(observation)
    observation, reward, done, info = env.step(action)
    
    score += reward
    
    if done:
        observation = env.reset()
        
env.close()
print(score)

In [ ]:
# model = ARS(3, 1, b=N_ROLLOUTS, std=1, 
#             state_low=state_low, state_high=state_high)

# REWARDS = []
# t_reward = 0
# for step in tqdm(range(TOTAL_STEPS)):
#     #env.render()
    
#     for ro in range(N_ROLLOUTS):
#         pos_action, neg_action, noise = model.get_actions(observation, is_train=True)
#         pos_observation, pos_reward, pos_done, pos_info = env.step(pos_action)
#         neg_observation, neg_reward, neg_done, neg_info = env.step(neg_action)

#         model.remember([pos_action[0], neg_action[0], noise, pos_reward, neg_reward])
#         t_reward += (pos_reward + neg_reward)/2
#         #observation = env.reset()
#         observation = pos_observation if np.random.rand()>0.5 else neg_observation

#     model.learn()
#     REWARDS.append(t_reward)
#     t_reward = 0
    
# env.close()

In [ ]:
# model.policy(np.array([1,1,0]))